In [1]:
%pip install tableone

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import tableone
from google.colab import files
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

%load_ext google.colab.data_table

Authenticated


In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/SCCM Datathon 2023/Code

/content/drive/MyDrive/SCCM Datathon 2023/Code


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Get the data

from google.cloud import bigquery
client = bigquery.Client("sccm-datathon-2023-participant")


sql = """
WITH sorted_data AS (
  SELECT
    *,
    icustays.subject_id ,
    icustays.stay_id as qwe,
    ROW_NUMBER() OVER (PARTITION BY icustays.subject_id, icustays.stay_id ORDER BY vent.starttime) as row_num
  FROM `physionet-data.mimiciv_derived.ventilation` vent
  JOIN `physionet-data.mimiciv_icu.icustays` icustays ON vent.stay_id = icustays.stay_id
  JOIN  `physionet-data.mimiciv_hosp.patients` patients on icustays.subject_id = patients.subject_id
  JOIN  `physionet-data.mimiciv_hosp.admissions` admissions on admissions.subject_id = patients.subject_id
  JOIN `physionet-data.mimiciv_icu.chartevents` events on events.stay_id = icustays.stay_id

  WHERE vent.ventilation_status = "InvasiveVent"
  AND timestamp_diff(vent.endtime, vent.starttime, HOUR) >= 48

)
SELECT *
FROM sorted_data sd
JOIN `physionet-data.mimiciv_icu.inputevents` inputevents ON inputevents.stay_id = sd.qwe
--JOIN `physionet-data.mimiciv_icu.d_items` ditems ON inputevents.itemid = sd.itemid
WHERE sd.row_num = 1
AND inputevents.itemid IN (221744, 221833, 225154, 221385, 221668, 222168, 225150, 229420)
AND inputevents.rate IS NOT NULL
"""

res = client.query(sql).to_dataframe()

In [ ]:
from google.cloud import bigquery
client = bigquery.Client("sccm-datathon-2023-participant")


sql = """
WITH sorted_data AS (
  SELECT
    *,
    icustays.subject_id ,
    icustays.stay_id as qwe,
    --FIRST_VALUE(*) OVER (PARTITION BY icustays.subject_id, icustays.stay_id ORDER BY vent.starttime) AS firstValue
    ROW_NUMBER() OVER (PARTITION BY icustays.subject_id, icustays.stay_id ORDER BY vent.starttime) as row_num
  FROM `physionet-data.mimiciv_derived.ventilation` vent
  LEFT JOIN `physionet-data.mimiciv_icu.icustays` icustays ON vent.stay_id = icustays.stay_id
  LEFT JOIN  `physionet-data.mimiciv_hosp.patients` patients on icustays.subject_id = patients.subject_id
  LEFT JOIN  `physionet-data.mimiciv_hosp.admissions` admissions on admissions.subject_id = patients.subject_id
  LEFT JOIN `physionet-data.mimiciv_icu.chartevents` events on events.stay_id = icustays.stay_id

  WHERE vent.ventilation_status = "InvasiveVent"
  AND timestamp_diff(vent.endtime, vent.starttime, HOUR) >= 48
),

medication AS (
  SELECT
  MIN(inputevents.starttime) AS drug_starttime,
  inputevents.itemid AS drug_id,
  inputevents.stay_id,
  FIRST_VALUE (MIN(inputevents.starttime))
  OVER (PARTITION BY inputevents.stay_id ORDER BY MIN(inputevents.starttime) ASC)
  FROM `physionet-data.mimiciv_icu.inputevents` inputevents
  WHERE inputevents.itemid IN
  (221744, 221833, 225154, 221385, 221668, 222168, 225150, 229420)
  AND rate IS NOT NULL
  GROUP BY inputevents.stay_id, inputevents.itemid
)
SELECT
*
FROM sorted_data sd
LEFT JOIN medication as med ON med.stay_id = sd.qwe
WHERE sd.row_num = 1

"""

res = client.query(sql).to_dataframe()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred

In [ ]:
# Convert 'Category' column to dummy variables
dummy_df = pd.get_dummies(res['drug_id'])

# Concatenate the dummy DataFrame with the original DataFrame
df = pd.concat([res, dummy_df], axis=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df = df.drop_duplicates()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
for column in df.columns:
  print(column)

stay_id
starttime
endtime
ventilation_status
subject_id
hadm_id
stay_id_1
first_careunit
last_careunit
intime
outtime
los
subject_id_1
gender
anchor_age
anchor_year
anchor_year_group
dod
subject_id_2
hadm_id_1
admittime
dischtime
deathtime
admission_type
admit_provider_id
admission_location
discharge_location
insurance
language
marital_status
race
edregtime
edouttime
hospital_expire_flag
subject_id_3
hadm_id_2
stay_id_2
caregiver_id
charttime
storetime
itemid
value
valuenum
valueuom
warning
subject_id_4
qwe
row_num
drug_starttime
drug_id
stay_id_3
f0_
221385
221668
221744
221833
222168
225150
225154
229420


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df.columns

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index([             'stay_id',            'starttime',              'endtime',
         'ventilation_status',           'subject_id',              'hadm_id',
                  'stay_id_1',       'first_careunit',        'last_careunit',
                     'intime',              'outtime',                  'los',
               'subject_id_1',               'gender',           'anchor_age',
                'anchor_year',    'anchor_year_group',                  'dod',
               'subject_id_2',            'hadm_id_1',            'admittime',
                  'dischtime',            'deathtime',       'admission_type',
          'admit_provider_id',   'admission_location',   'discharge_location',
                  'insurance',             'language',       'marital_status',
                       'race',            'edregtime',            'edouttime',
       'hospital_expire_flag',         'subject_id_3',            'hadm_id_2',
                  'stay_id_2',         'caregiver_id

In [ ]:
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,stay_id,starttime,endtime,ventilation_status,subject_id,hadm_id,stay_id_1,first_careunit,last_careunit,intime,...,stay_id_3,f0_,221385,221668,221744,221833,222168,225150,225154,229420
0,37864255,2168-05-15 16:00:00,2168-05-18 18:00:00,InvasiveVent,11067215,23574374,37864255,Trauma SICU (TSICU),Medical Intensive Care Unit (MICU),2168-05-15 14:20:52,...,37864255,2168-05-15 10:30:00,0,0,0,0,1,0,0,0
1,37864255,2168-05-15 16:00:00,2168-05-18 18:00:00,InvasiveVent,11067215,23574374,37864255,Trauma SICU (TSICU),Medical Intensive Care Unit (MICU),2168-05-15 14:20:52,...,37864255,2168-05-15 10:30:00,0,0,1,0,0,0,0,0
2,37864255,2168-05-15 16:00:00,2168-05-18 18:00:00,InvasiveVent,11067215,23574374,37864255,Trauma SICU (TSICU),Medical Intensive Care Unit (MICU),2168-05-15 14:20:52,...,37864255,2168-05-15 10:30:00,0,1,0,0,0,0,0,0
3,35363753,2123-01-16 18:00:00,2123-01-19 20:56:00,InvasiveVent,15069820,25463001,35363753,Trauma SICU (TSICU),Trauma SICU (TSICU),2123-01-16 17:20:00,...,35363753,2123-01-16 17:30:00,0,0,0,0,1,0,0,0
4,36095830,2117-10-21 12:00:00,2117-10-24 11:00:00,InvasiveVent,17349939,20829317,36095830,Trauma SICU (TSICU),Trauma SICU (TSICU),2117-10-21 12:02:00,...,36095830,2117-10-21 12:34:00,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15285,32903465,2123-09-25 23:18:00,2123-09-27 23:00:00,InvasiveVent,13261551,22549722,32903465,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Surgical Intensive Care Unit (Neuro SICU),2123-09-25 23:09:00,...,32903465,2123-09-25 23:30:00,0,0,0,0,1,0,0,0
15286,31122178,2170-07-09 03:06:00,2170-07-25 16:00:00,InvasiveVent,18571536,20050336,31122178,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2170-07-09 02:56:00,...,31122178,2170-07-09 03:00:00,0,1,0,0,0,0,0,0
15287,31122178,2170-07-09 03:06:00,2170-07-25 16:00:00,InvasiveVent,18571536,20050336,31122178,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2170-07-09 02:56:00,...,31122178,2170-07-09 03:00:00,0,0,0,0,1,0,0,0
15288,31122178,2170-07-09 03:06:00,2170-07-25 16:00:00,InvasiveVent,18571536,20050336,31122178,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2170-07-09 02:56:00,...,31122178,2170-07-09 03:00:00,0,0,1,0,0,0,0,0


In [ ]:
df_aux = df.groupby(['stay_id', 'subject_id']).agg(
    {
        'starttime': 'min',
        'endtime': 'min',
        'gender': 'min',
        'race': 'min',
        'language': 'min',
        'insurance': 'min',

        221385: 'sum',
        221668: 'sum',
        221744: 'sum',
        221833: 'sum',
        222168: 'sum',
        225150: 'sum',
        225154: 'sum',
        229420: 'sum',

    }
).reset_index().drop_duplicates(subset='subject_id', keep='first').drop_duplicates(subset='stay_id', keep='first')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df_aux

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
df_aux.to_csv('Final_Cohort.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
